# Outline
1. How programs organize memory
2. Memory management 
3. Garbage Collection

## How programs organize memory

Memory is allocated on the stack or on the heap. When memory is allocated on the stack, it is freed when a stack frame is "popped" or removed from the stack. This happens automatically as the program executes. On the other hand, when a programmer wants to allocate a large amount of memory or needs to allocate dynamically (perhaps the amount of memory required is determined by the program execution), the programmer will use the heap. See https://en.wikipedia.org/wiki/Stack-based_memory_allocation and https://en.wikipedia.org/wiki/Memory_management#DYNAMIC for more.

If all references to a block of dynamically allocated memory are lost, your program has a "memory leak," which means that the block cannot be accessed or reallocated by your program. Most often, when the program finishes executing, your Operating System will free any memory leaked. Another problem that can occur happens when memory is freed or reallocated, but a reference to the original memory is deferenced after the fact. This is called a "dangling pointer."

Memory management can be done by the programmer (self-management) or it can be done automatically by the program runtime. Let's take a look at how self-management can be implemented in Lettuce.


## Let's add a `DeleteRef` expression that will remove a `Reference` from the `Store`.


In [ ]:
sealed trait Program
sealed trait Expr

case class TopLevel(e: Expr) extends Program

case class Const(v: Double) extends Expr
case class Ident(s: String) extends Expr

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr

// If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

// Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

// Function definition
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

// New Ref
case class NewRef(e: Expr) extends Expr

// DeRef
case class DeRef(lval: Expr) extends Expr

// AssignRef
case class AssignRef(lval: Expr, rval: Expr) extends Expr

//BEGIN SOLUTION
case class DeleteRef(e: Expr) extends Expr
//END SOLUTION

## We also need to implement the side effect of evaluating `DeleteRef`: deleting the cell corresponding to the reference passed.

In [ ]:
sealed trait Value
case class NumValue(f: Double) extends Value
case class BoolValue(b: Boolean) extends Value
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value 
case class Reference(j: Int) extends Value
case object ErrorValue extends Value

case object NullValue extends Value


def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}


case class ImmutableStore(val storeMap: Map[Int, Value])
    
def createNewCell(s: ImmutableStore, v: Value): (ImmutableStore, Int) = {
        /*- make a new cell -*/
        val newAddr = s.storeMap.size
        val newMap = s.storeMap + (newAddr -> v)
        val newStore = ImmutableStore(newMap) // Make a new store with one more cell
        (newStore, newAddr)
}
    
def lookupCellValue(s: ImmutableStore, j: Int): Value = {
        if (s.storeMap.contains(j)){
            s.storeMap(j)
        } else {
            throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j")
        }
}

def assignToCell(s: ImmutableStore, j: Int, v: Value): ImmutableStore = {
    if (s.storeMap.contains(j)){
        val nMap = s.storeMap + (j -> v) // Update the store map.
        ImmutableStore(nMap)
    } else {
        throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
    }
}

def deleteCell(s: ImmutableStore, j: Int): ImmutableStore = {
    // BEGIN SOLUTION   
    if (s.storeMap.contains(j)){
        ImmutableStore(s.storeMap - j)
    } else {
        throw new IllegalArgumentException(s"Illegal delete of nonexistant location $j")
    }
    // END SOLUTION
}
    

## Now, when evaluating an expression, we can delete cell `j` when a sub-expression evaluates to `DeleteRef(Reference(j))`.

In [ ]:
def evalExpr(e: Expr, env: Map[String, Value], store: ImmutableStore): (Value, ImmutableStore) = {
      
    /* Method to deal with binary arithmetic operations */
    def applyArith2(e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (NumValue(v3), store2)
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val (v,store1) = evalExpr(e, env, store)
        val v1 = fun(valueToNumber(v))
        (NumValue(v1), store1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (BoolValue(v3), store2)
    }
    
    
    e match {
        case Const(f) => (NumValue(f), store)
        
        case Ident(x) => {
            if (env contains x) 
                (env(x), store)
            else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
            
        case Minus(e1, e2) => applyArith2(e1, e2) ( _ - _ )
    
        case Mult(e1, e2) =>  applyArith2(e1, e2) (_ * _)
        
        case Geq(e1, e2) => applyComp(e1, e2)(_ >= _)
    
        case Eq(e1, e2) => applyComp(e1, e2)(_ == _)
        
        case IfThenElse(e1, e2, e3) => {
            val (v, store1) = evalExpr(e1, env, store)
            v match {
                case BoolValue(true) => evalExpr(e2, env, store1)
                case BoolValue(false) => evalExpr(e3, env, store1)
                case _ => throw new IllegalArgumentException(s"If-then-else condition expr: ${e1} is non-boolean -- evaluates to ${v}")
            }
        }
        
        case Let(x, e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2, store1) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            (Closure(x, e, env), store) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            val (v2, store2) = evalExpr(e2, env, store1)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env, store2)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        
        case NewRef(e) => {
            val (v, store1) = evalExpr(e, env, store)
            val (store2, j) = createNewCell(store1, v)
            (Reference(j), store2)
        }
        
        case DeRef(e) => {
            val (v, store1) = evalExpr(e, env, store)
            v match {
                case Reference(j) => {
                    val v = lookupCellValue(store1, j)
                    (v, store1)
                }
                case _ => throw new IllegalArgumentException(s"Deref applied to an expression that does not evaluate to a reference")
            }
        }
        
        case AssignRef(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            v1 match {
                case Reference(j) => {
                    val (v2, store2) = evalExpr(e2, env, store1)
                    val store3 = assignToCell(store2, j, v2)
                    (v2, store3)
                }
                case _ => throw new IllegalArgumentException(s"AssignRef applied to argument that is not a reference")
                
            }
        }
        
        case DeleteRef(e) => {
            // BEGIN SOLUTION
            val (v, store1) = evalExpr(e, env, store)
            v match {
                case Reference(j) => {
                    val store2 = deleteCell(store1, j)
                    (NullValue, store2) 
                }
                case _ => throw new IllegalArgumentException(s"DeleteRef applied to argument that is not a reference")
            }
            // END SOLUTION
        }
        
    }

}

def evalProgram(p: Program) = p match {
        case TopLevel(e) => { 
            // Start with empty environment and empty store
            val (v1, s1) = evalExpr(e, Map(), new ImmutableStore(Map()))
            v1
        }
}
    

In [ ]:
/*
let x = NewRef(20) in 
let y = DeRef(x) + 1 in y
*/

val x = Ident("x")
val y = Ident("y")

val e = Let("x", NewRef(Const(20.0)), 
           Let("y", Plus(Const(1.0), DeRef(x)), 
              y))

println(s"Result: ${evalProgram(TopLevel(e))}")

In [ ]:
/*
let x = NewRef(10) in 
let dummy = DeleteRef(x) in
let y = DeRef(x) + 1 in y
*/

val x = Ident("x")
val y = Ident("y")

val e = Let("x", NewRef(Const(20.0)), 
           Let("dummy", DeleteRef(x), 
               Let("y", Plus(Const(1.0), DeRef(x)), y)))

println(s"Result: ${evalProgram(TopLevel(e))}")